# Lombscargle CSD

First part of this notebook looks at scipy versions which are a bit loose with the maths and don't preserve the complex nature of the periodogram. 

Second part is me defining my own lombscargle and testing effects with various irregularly sampled data

Results: Signal peaks are quite invariant but the noise floor does interesting things with lombscargle depending on which samples are missing from a dataset.

In [ ]:
import numpy
import matplotlib.pyplot as plt
from scipy.signal.spectral import lombscargle

# Test with Artificial Signal

Start with regularly sampled signal

In [ ]:
xvar = numpy.arange(0,128,1.0)

fft = numpy.zeros(shape=(128),dtype=numpy.complex128)
lmb = numpy.zeros(shape=(64))
# Test averaging
averages = 1000
for i in numpy.arange(averages):

    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128)/5 +numpy.sin(2*numpy.pi * 0.08 * xvar)

    #plt.plot(xvar,sine_test)
    #plt.show()

    hamm_sine = numpy.hamming(128) * sine_test
    ## FFT
    fft += numpy.fft.fft(hamm_sine)/128
    freqs = numpy.fft.fftfreq(len(fft))
    
    ## Lombscargle
    freqs = numpy.arange(0.0001,1.28,0.02)
    lmb += lombscargle(xvar, hamm_sine,freqs)

    
plt.semilogy(xvar,numpy.abs(fft/averages))
plt.semilogy(numpy.arange(0,64,1),numpy.abs(lmb/averages))


## Irregularly Sampled Signal

A single flagged out channel

In [ ]:
randn = numpy.random.randint(0,128)
fft = numpy.zeros(shape=(128),dtype=numpy.complex128)
lmb = numpy.zeros(shape=(64))
averages=1000
for i in numpy.arange(averages):

    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128)/5+numpy.sin(2*numpy.pi * 0.06 * xvar)
    hamm_sine = numpy.hamming(128) * sine_test
    hamm_sine_test_fft = hamm_sine.copy()
    
    hamm_sine_test_fft[randn] = 0.0
    hamm_sine_test_lmb = hamm_sine.copy()
    hamm_sine_test_lmb = numpy.delete(hamm_sine_test_lmb,randn)
    xvar_lmb = xvar.copy()
    xvar_lmb = numpy.delete(xvar_lmb, randn)
    #plt.plot(xvar,sine_test)
    #plt.show()

    
    ## FFT
    fft += numpy.fft.fft(hamm_sine_test_fft)/128
    freqs = numpy.fft.fftfreq(len(fft))
    
    ## Lombscargle
    freqs = numpy.arange(0.0001,0.64,0.01)
    lmb += lombscargle(xvar_lmb, hamm_sine_test_lmb,freqs)/128

    
plt.semilogy(xvar,numpy.square(numpy.abs(fft/averages)))    
plt.semilogy(numpy.arange(0,64,1),numpy.abs(lmb/averages))

# Scipy Lombscargle doesn't look quite right..

Try doing it manually. Start with classical periodogram.

In [ ]:
def classical_periodogram(x,y,freqs):
    #x = x.astype(int)
    out = numpy.zeros(shape=freqs.shape[0],dtype=numpy.complex64)
    for i in range(freqs.shape[0]):
        
        xe = 0.0 + 1j * 0.0
        
        
        for j in range(x.shape[0]):
            
            e = numpy.exp(1j * freqs[i] * x[j])
            xe += y[j] * e
        #print xe
        out[i] = xe / y.shape[0]
    return out
            

In [ ]:
averages = 1000
freqs = numpy.arange(0.0001,0.64,0.01)
xvar = numpy.arange(0.0,128.0,1.0)
agg_periodogram = numpy.zeros(freqs.shape[0],dtype=numpy.complex128)
agg_fft = numpy.zeros(xvar.shape[0],dtype=numpy.complex128)

for i in numpy.arange(averages):
    
    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128)/5+numpy.sin(2*numpy.pi * 0.06 * xvar)
    hamm_sine = numpy.hamming(sine_test.shape[0]) * sine_test
    
    agg_fft += numpy.fft.fft(hamm_sine)/128
    agg_periodogram += classical_periodogram(xvar,hamm_sine,freqs)
    
print(agg_periodogram)
plt.semilogy(numpy.arange(0,64,1),numpy.abs(agg_periodogram/averages))
plt.semilogy(xvar,numpy.abs(agg_fft/averages))
    

Much better, now try doing it with lombscargle..

In [ ]:
# As described in equation 33 off VanderPlas - Understanding the Lomb-Scargle Periodogram
def my_lombscargle(x,y,freqs):
    #x = x.astype(int)
    out = numpy.zeros(shape=freqs.shape[0],dtype=numpy.complex128)
    for i in range(freqs.shape[0]):
        
        tau = 0.0
        sin = 0.0
        cos = 0.0
        
        for j in range(x.shape[0]):
            
            sin += numpy.sin(4 * numpy.pi * freqs[i] * x[j])
            cos += numpy.cos(4 * numpy.pi * freqs[i] * x[j])
            
        tau = numpy.arctan(sin/cos)
        tau = tau * (1/(4 * numpy.pi * freqs[i]))
        #print(tau)
        a_num = 0.0
        a_denom = 0.0
        
        b_num = 0.0
        b_denom = 0.0
        
        
            
        for j in range(x.shape[0]):
            
            a_num += y[j] * numpy.cos(2 * numpy.pi * freqs[i]*(x[j] - tau))
            a_denom += numpy.square(numpy.cos(2 * numpy.pi * freqs[i]*(x[j] - tau)))
            
            b_num += y[j] * numpy.sin(2 * numpy.pi * freqs[i]*(x[j] - tau))
            b_denom += numpy.square(numpy.sin(2 * numpy.pi * freqs[i]*(x[j] - tau)))
            
            
        comp_out = (a_num/a_denom) + 1j*(b_num/b_denom)
            
        #print xe
        out[i] = comp_out / 2
    return out

Test with regularly sampled data firstly.

In [ ]:
averages = 10
freqs = numpy.arange(0.0001,0.16,0.00025)
xvar = numpy.arange(0.0,128.0,1.0)
agg_periodogram = numpy.zeros(freqs.shape[0],dtype=numpy.complex128)
agg_fft = numpy.zeros(xvar.shape[0],dtype=numpy.complex128)
agg_lmb = numpy.zeros(freqs.shape[0],dtype=numpy.complex128)
for i in numpy.arange(averages):
    
    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128)/5+numpy.sin(2*numpy.pi * 0.08 * xvar)
    hamm_sine = numpy.hamming(sine_test.shape[0]) * sine_test
    
    agg_fft += numpy.fft.fft(hamm_sine)/128
    agg_periodogram += classical_periodogram(xvar, hamm_sine,freqs)
    agg_lmb += my_lombscargle(xvar,hamm_sine,freqs)
    
#print(agg_periodogram)

plt.semilogy(freqs,numpy.abs(agg_lmb)/averages)
plt.semilogy(freqs,numpy.abs(agg_periodogram)/averages)
#plt.semilogy(xvar,numpy.abs(agg_fft/averages))

# Test with Irregularly Sampled Data

Here we remove 3 random samples for 3 different runs to look at effects. 

Result: Signal peaks seem quite invariant. Noise does weird stuff. 

In [ ]:
for k in numpy.arange(3):

    averages = 10
    freqs = numpy.arange(0.0001,0.16,0.00025)
    xvar = numpy.arange(0.0,128.0,1.0)

    agg_lmb = numpy.zeros(freqs.shape[0],dtype=numpy.complex128)


    smp = 1
    randn = numpy.random.choice(numpy.arange(0,128),smp,replace=False)
    print(randn.shape)
    print(randn)
    xvar = numpy.delete(xvar,randn)
    print(xvar.shape)

    for i in numpy.arange(averages):
    
        sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128-smp)/5+numpy.sin(2*numpy.pi * 0.08 * xvar)
        hamm_sine = numpy.hamming(sine_test.shape[0]) * sine_test
    
        agg_lmb += my_lombscargle(xvar,hamm_sine,freqs)
    
#print(agg_periodogram)



    plt.semilogy(freqs,numpy.abs(agg_lmb))


# Lombscargle Cross-Spectral Density

Artificial Example. Two Sine Signals

In [ ]:
averages = 10
freqs = numpy.arange(0.0001,0.16,0.00025)
xvar = numpy.arange(0.0,128.0,1.0)

agg_lmb = numpy.zeros(freqs.shape[0],dtype=numpy.complex128)
agg_lmb2 = numpy.zeros(freqs.shape[0],dtype=numpy.complex128)

randn = numpy.random.choice(numpy.arange(0,128),40,replace=False)
print(randn.shape)
print(randn)
xvar = numpy.delete(xvar,randn)
print(xvar.shape)

for i in numpy.arange(averages):
    
    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128-40)/5+numpy.sin(2*numpy.pi * 0.08 * xvar)
    sine_test2 = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128-40)/5+numpy.sin(2*numpy.pi * 0.06 * xvar)
    hamm_sine = numpy.hamming(sine_test.shape[0]) * sine_test
    hamm_sine2 = numpy.hamming(sine_test2.shape[0]) * sine_test2
    agg_lmb += my_lombscargle(xvar,hamm_sine,freqs)
    agg_lmb2 += my_lombscargle(xvar,hamm_sine2,freqs)
    
    
#print(agg_periodogram)
plt.semilogy(freqs,numpy.abs(agg_lmb))
plt.semilogy(freqs,numpy.abs(agg_lmb2))
plt.semilogy(freqs,numpy.abs(agg_lmb * numpy.conjugate(agg_lmb2)))

